In [1]:
from leeq.core.elements.built_in.qudit_transmon import TransmonElement
from leeq.setups.built_in.setup_simulation_high_level import HighLevelSimulationSetup
import numpy as np
from leeq.theory.simulation.numpy.rotated_frame_simulator import VirtualTransmon
from leeq import ExperimentManager
from labchronicle import Chronicle
Chronicle().start_log()
manager = ExperimentManager()
manager.clear_setups()

virtual_transmon_1 = VirtualTransmon(
    name="VQubit_1",
    qubit_frequency=5040.4,
    anharmonicity=-198,
    t1=70,
    t2=35,
    readout_frequency=9645.5,
    quiescent_state_distribution=np.asarray(
        [
            0.8,
            0.15,
            0.04,
            0.01]))

virtual_transmon_2 = VirtualTransmon(
    name="VQubit_2",
    qubit_frequency=5040. - 123,
    anharmonicity=-197,
    t1=70,
    t2=35,
    readout_frequency=9645.5 + 100,
    quiescent_state_distribution=np.asarray(
        [
            0.8,
            0.15,
            0.04,
            0.01]))

setup = HighLevelSimulationSetup(
    name='HighLevelSimulationSetup',
    virtual_qubits={2: virtual_transmon_1,
                    4: virtual_transmon_2
                    }
)
setup.set_coupling_strength_by_qubit(
    virtual_transmon_1, virtual_transmon_2, coupling_strength=1.5)

manager.register_setup(setup)

configuration_q1 = {
    'lpb_collections': {
        'f01': {
            'type': 'SimpleDriveCollection',
            'freq': 5040.4,
            'channel': 2,
            'shape': 'blackman_drag',
            'amp': 0.21323904814245054 / 5 * 4,
            'phase': 0.,
            'width': 0.025,
            'alpha': 425.1365229849309,
            'trunc': 1.2
        },
        'f12': {
            'type': 'SimpleDriveCollection',
            'freq': 5040.4 - 198,
            'channel': 2,
            'shape': 'blackman_drag',
            'amp': 0.21323904814245054 / 5 * 4,
            'phase': 0.,
            'width': 0.025,
            'alpha': 425.1365229849309,
            'trunc': 1.2
        }
    },
    'measurement_primitives': {
        '0': {
            'type': 'SimpleDispersiveMeasurement',
            'freq': 9645.5,
            'channel': 1,
            'shape': 'square',
            'amp': 0.21323904814245054 / 5 * 4,
            'phase': 0.,
            'width': 1,
            'trunc': 1.2,
            'distinguishable_states': [0, 1]
        },
        '1': {
            'type': 'SimpleDispersiveMeasurement',
            'freq': 9144.41,
            'channel': 1,
            'shape': 'square',
            'amp': 0.21323904814245054 / 5 * 4,
            'phase': 0.,
            'width': 1,
            'trunc': 1.2,
            'distinguishable_states': [0, 1, 2]
        }
    }
}

configuration_q2 = {
    'lpb_collections': {
        'f01': {
            'type': 'SimpleDriveCollection',
            'freq': 5040. - 123,
            'channel': 4,
            'shape': 'blackman_drag',
            'amp': 0.21323904814245054 / 5 * 4,
            'phase': 0.,
            'width': 0.025,
            'alpha': 425.1365229849309,
            'trunc': 1.2
        },
        'f12': {
            'type': 'SimpleDriveCollection',
            'freq': 5040.4 - 198 - 123,
            'channel': 4,
            'shape': 'blackman_drag',
            'amp': 0.21323904814245054 / 5 * 4,
            'phase': 0.,
            'width': 0.025,
            'alpha': 425.1365229849309,
            'trunc': 1.2
        }
    },
    'measurement_primitives': {
        '0': {
            'type': 'SimpleDispersiveMeasurement',
            'freq': 9645.5 + 100,
            'channel': 3,
            'shape': 'square',
            'amp': 0.21323904814245054 / 5 * 4,
            'phase': 0.,
            'width': 1,
            'trunc': 1.2,
            'distinguishable_states': [0, 1]
        },
        '1': {
            'type': 'SimpleDispersiveMeasurement',
            'freq': 9645.5 + 100,
            'channel': 3,
            'shape': 'square',
            'amp': 0.21323904814245054 / 5 * 4,
            'phase': 0.,
            'width': 1,
            'trunc': 1.2,
            'distinguishable_states': [0, 1, 2]
        }
    }
}

qubit_1 = TransmonElement(
        name='test-qubit-1',
        parameters=configuration_q1
    )

qubit_2 = TransmonElement(
        name='test-qubit-2',
        parameters=configuration_q2
    )

[2024-10-01 20:37:48] [INFO] [labchronicle.chronicle] Log started at log/zijian/2024-10/2024-10-01/20.37.48
[2024-10-01 20:37:48] [INFO] [labchronicle.chronicle] Log started at log/zijian/2024-10/2024-10-01/20.37.48


In [4]:
import os
from k_agents.execution.agent import AutoRun
from k_agents.translation.env import TranslationAgentEnv
from leeq.utils.ai.translation_agent import init_leeq_translation_agent
from leeq.experiments import experiments as exp
from mllm import config, caching

#config.default_models.normal = "chatgpt-4o-latest"
#config.default_models.expensive = "chatgpt-4o-latest"
config.default_models.normal = "gpt-4o"
config.default_models.expensive = "gpt-4o"
config.default_options.temperature = 0.2
root = os.path.dirname(exp.__file__)
init_leeq_translation_agent()
env = TranslationAgentEnv()

ExperimentManager().status().set_param("Plot_Result_In_Jupyter", True)
duts = [qubit_1, qubit_2]
frequency1 = duts[0].get_c1('f01').get_parameters()["freq"]

#AutoRun("""
#- Run a Two-qubit calibration on `duts`
#""", frequency=frequency1, duts=duts)
with caching.refresh_cache():
    AutoRun(
        "Two level Two-qubit calibration on `duts`", duts=duts
    )

Adding experiment to memory:   0%|          | 0/17 [00:00<?, ?it/s]

generate_idea_from_procedure:   0%|          | 0/7 [00:00<?, ?it/s]

_get_stage_from_agent_response:   0%|          | 0/3 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

Recalling:   0%|          | 0/5 [00:00<?, ?it/s]

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>